<a href="https://colab.research.google.com/github/Samin-Sadaf7/Image_works/blob/main/VideoToImageProcessingWebCrawlingForDatasetCreation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
!pip install pytube opencv-python torch torchvision matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.9 MB/s eta 0:00:00


In [4]:
import os
import cv2
import torch
import pytube
from pytube import YouTube
import matplotlib.pyplot as plt

In [5]:
VIDEO_DIR = "/content/gdrive/MyDrive/VideoToImage_TrafficSign/traffic_sign_videos"
FRAME_DIR = "/content/gdrive/MyDrive/VideoToImage_TrafficSign/traffic_sign_frames"
os.makedirs(VIDEO_DIR, exist_ok=True)
os.makedirs(FRAME_DIR, exist_ok=True)

In [6]:
!pip --quiet install pytubefix

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 1.8 MB/s eta 0:00:00


In [7]:
from pytubefix import YouTube
from pytubefix.cli import on_progress
import os

def download_youtube_video(url, save_path=VIDEO_DIR):
    os.makedirs(save_path, exist_ok=True)
    yt = YouTube(url, on_progress_callback=on_progress)
    video_title = yt.title
    print(f"Downloading video: {video_title}")
    ys = yt.streams.get_highest_resolution()
    video_path = ys.download(output_path=save_path)
    print(f"Video downloaded to: {video_path}")
    return video_path, video_title

In [8]:
!sudo apt-get update
!sudo apt-get install libgtk2.0-dev pkg-config

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,172 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,498 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-sec

In [104]:
import cv2
import os

def extract_frames(video_path, video_title, output_dir, frame_interval=10):
    """
    Extracts high-quality frames from a video file and saves them as PNG.

    Parameters:
    - video_path (str): Path to the video file.
    - video_title (str): Title of the video, used in naming the frames.
    - output_dir (str): Directory to save the extracted frames.
    - frame_interval (int): Extract a frame every 'frame_interval' frames.
    """
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Open the video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Unable to open video file: {video_path}")
        return

    # Get video properties
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_rate = cap.get(cv2.CAP_PROP_FPS)

    frame_number = 0
    extracted_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Only process frames at specified intervals
        if frame_number % frame_interval == 0:
            # Calculate timestamp and sanitize title for the filename
            timestamp = frame_number / frame_rate
            timestamp_str = f"{timestamp:.2f}s"
            sanitized_title = video_title.replace(" ", "_").replace("/", "_")

            # Construct frame filename
            frame_filename = os.path.join(output_dir, f"{sanitized_title}_frame_{timestamp_str}.png")

            # Save the frame in PNG format (high quality, minimal compression)
            cv2.imwrite(frame_filename, frame, [cv2.IMWRITE_PNG_COMPRESSION, 0])
            extracted_count += 1
            print(f"Frame saved: {frame_filename}")

        frame_number += 1

    # Release the video capture object
    cap.release()

    print(f"Total frames extracted: {extracted_count}")

In [109]:
import os
import re

def time_to_seconds(minutes, seconds):
    """
    Converts minutes and seconds to total seconds.

    Parameters:
        minutes (int): The number of minutes.
        seconds (int): The number of seconds.

    Returns:
        float: Total time in seconds.
    """
    return minutes * 60 + seconds

def delete_frames_in_time_range(output_dir, start_time_minutes, start_time_seconds, end_time_minutes, end_time_seconds):
    """
    Deletes frames within the specified time range.

    Parameters:
        output_dir (str): Directory where frames are stored.
        start_time_minutes (int): Start time in minutes.
        start_time_seconds (int): Start time in seconds.
        end_time_minutes (int): End time in minutes.
        end_time_seconds (int): End time in seconds.
    """
    # Convert time to seconds
    start_time = time_to_seconds(start_time_minutes, start_time_seconds)
    end_time = time_to_seconds(end_time_minutes, end_time_seconds)

    # Ensure the time range is valid
    if start_time >= end_time:
        print("Error: Start time must be less than end time.")
        return

    # Compile a regex pattern to extract timestamps from filenames
    timestamp_pattern = re.compile(r"_(\d+\.\d+)s\.png$")

    deleted_count = 0

    # Iterate over all files in the output directory
    for filename in os.listdir(output_dir):
        match = timestamp_pattern.search(filename)
        if match:
            # Extract the timestamp from the filename
            timestamp = float(match.group(1))
            # Check if the timestamp falls within the specified range
            if start_time <= timestamp <= end_time:
                file_path = os.path.join(output_dir, filename)
                os.remove(file_path)
                deleted_count += 1
                print(f"Deleted frame: {filename}")

    print(f"Total frames deleted: {deleted_count}")

In [107]:
video_urls = ["https://www.youtube.com/watch?v=-Fp-n5dh79g&pp=ygUKcm9hZCB0dWJlcg%3D%3D"]
output_dir =  '/content/gdrive/MyDrive/VideoToImage_TrafficSign/traffic_sign_frames/RoadTuber_going_from_Sylhet_toLamakazi'
for url in video_urls:
      video_path, video_title = download_youtube_video(url)
      extract_frames(video_path, video_title, output_dir)

Video downloaded to: /content/gdrive/MyDrive/VideoToImage_TrafficSign/traffic_sign_videos/Sylhet Biswanath to Lamakazi via Rampasha 4K | Scenic Drive | Road Tuber | Sylhet Bangladesh.mp4
Frame saved: /content/gdrive/MyDrive/VideoToImage_TrafficSign/traffic_sign_frames/RoadTuber_going_from_Sylhet_toLamakazi/Sylhet_Biswanath_to_Lamakazi_via_Rampasha_4K_|_Scenic_Drive_|_Road_Tuber_|_Sylhet_Bangladesh_frame_0.00s.png
Frame saved: /content/gdrive/MyDrive/VideoToImage_TrafficSign/traffic_sign_frames/RoadTuber_going_from_Sylhet_toLamakazi/Sylhet_Biswanath_to_Lamakazi_via_Rampasha_4K_|_Scenic_Drive_|_Road_Tuber_|_Sylhet_Bangladesh_frame_0.33s.png
Frame saved: /content/gdrive/MyDrive/VideoToImage_TrafficSign/traffic_sign_frames/RoadTuber_going_from_Sylhet_toLamakazi/Sylhet_Biswanath_to_Lamakazi_via_Rampasha_4K_|_Scenic_Drive_|_Road_Tuber_|_Sylhet_Bangladesh_frame_0.67s.png
Frame saved: /content/gdrive/MyDrive/VideoToImage_TrafficSign/traffic_sign_frames/RoadTuber_going_from_Sylhet_toLamakazi/Sy

In [131]:
# Specify the time range for deleting frames
start_time_minutes = 20# Start time: minutes
start_time_seconds = 44 # Start time:  seconds
end_time_minutes = 25 # End time: minute
end_time_seconds = 0  # End time: seconds

# Call the function to delete frames within the specified range
delete_frames_in_time_range(
    output_dir,
    start_time_minutes, start_time_seconds,
    end_time_minutes, end_time_seconds
)

Deleted frame: Sylhet_Biswanath_to_Lamakazi_via_Rampasha_4K_|_Scenic_Drive_|_Road_Tuber_|_Sylhet_Bangladesh_frame_1244.00s.png
Deleted frame: Sylhet_Biswanath_to_Lamakazi_via_Rampasha_4K_|_Scenic_Drive_|_Road_Tuber_|_Sylhet_Bangladesh_frame_1244.33s.png
Deleted frame: Sylhet_Biswanath_to_Lamakazi_via_Rampasha_4K_|_Scenic_Drive_|_Road_Tuber_|_Sylhet_Bangladesh_frame_1244.67s.png
Deleted frame: Sylhet_Biswanath_to_Lamakazi_via_Rampasha_4K_|_Scenic_Drive_|_Road_Tuber_|_Sylhet_Bangladesh_frame_1245.00s.png
Deleted frame: Sylhet_Biswanath_to_Lamakazi_via_Rampasha_4K_|_Scenic_Drive_|_Road_Tuber_|_Sylhet_Bangladesh_frame_1245.33s.png
Deleted frame: Sylhet_Biswanath_to_Lamakazi_via_Rampasha_4K_|_Scenic_Drive_|_Road_Tuber_|_Sylhet_Bangladesh_frame_1245.67s.png
Deleted frame: Sylhet_Biswanath_to_Lamakazi_via_Rampasha_4K_|_Scenic_Drive_|_Road_Tuber_|_Sylhet_Bangladesh_frame_1246.00s.png
Deleted frame: Sylhet_Biswanath_to_Lamakazi_via_Rampasha_4K_|_Scenic_Drive_|_Road_Tuber_|_Sylhet_Bangladesh_fra

In [35]:
import os

def count_files_in_directory(directory_path):
    """
    Counts the total number of files in a given directory.

    Parameters:
        directory_path (str): Path to the directory.

    Returns:
        int: Total number of files in the directory.
    """
    if not os.path.exists(directory_path):
        print(f"Error: Directory {directory_path} does not exist.")
        return 0

    total_files = sum([len(files) for _, _, files in os.walk(directory_path)])
    return total_files

In [132]:
total_files = count_files_in_directory(output_dir)
print(f"Total number of files in the directory: {total_files}")

Total number of files in the directory: 150


In [71]:
import os
import shutil

def empty_directory(directory_path):
    if os.path.exists(directory_path) and os.path.isdir(directory_path):
        for filename in os.listdir(directory_path):
            file_path = os.path.join(directory_path, filename)
            try:
                if os.path.isfile(file_path) or os.path.islink(file_path):
                    os.remove(file_path)
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)
            except Exception as e:
                print(f"Failed to delete {file_path}. Reason: {e}")
    else:
        print("Directory does not exist or is not a directory.")

In [133]:
directory_to_empty = FRAME_DIR
empty_directory(directory_to_empty)

In [134]:
directory_to_empty = VIDEO_DIR
empty_directory(directory_to_empty)

In [72]:
directory_to_empty = output_dir
empty_directory(directory_to_empty)

Web Scrapping

In [ ]:
!pip install google_images_download

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00
  Created wheel for google_images_download: filename=google_images_download-2.8.0-py2.py3-none-any.whl size=14537 sha256=d80b2be9b9a3d7455ae212041e57349573fb6b6c60c28ac5503dee0e41bb840c
  Stored in directory: /root/.cache/pip/wheels/5b/f2/64/0eecb22fee5cbc0321d332492ddcc45b03c6373b5616ee95c8
Successfully built google_images_download


In [ ]:
!pip install icrawler

In [ ]:
from icrawler.builtin import GoogleImageCrawler
import os

In [ ]:
traffic_signs = [
    "Bangladesh Speed Limit 60 km/h sign on road in Bangladesh",
    "Bangladesh School traffic sign on road in Bangladesh",
    "Bangladesh Pedestrian Crossing traffic sign on road in Bangladesh",
    "Bangladesh Sharp Bend To The Left traffic sign on road in Bangladesh",
    "Bangladesh U-Turn traffic sign on road in Bangladesh",
    "Bangladesh Speed Limit 80 km/h sign on road in Bangladesh",
    "Bangladesh No Overtaking traffic sign on road in Bangladesh",
    "Bangladesh Sharp Bend To The Right traffic sign on road in Bangladesh",
    "Bangladesh Side Road Left traffic sign on road in Bangladesh",
    "Bangladesh Side Road Right traffic sign on road in Bangladesh",
    "Bangladesh Speed Limit 40 km/h sign on road in Bangladesh",
    "Bangladesh Narrow Bridge traffic sign on road in Bangladesh",
    "Bangladesh Road Hump traffic sign on road in Bangladesh",
    "Bangladesh No Use of Horn traffic sign on road in Bangladesh",
    "Bangladesh Crossroads traffic sign on road in Bangladesh"
]

In [ ]:
# Define the base output directory for saving images
base_output_dir = "/content/gdrive/MyDrive/VideoToImage_TrafficSign/traffic_signs_images"

In [ ]:
def download_traffic_sign_images(signs, limit=5):
    # Ensure the base directory exists
    if not os.path.exists(base_output_dir):
        os.makedirs(base_output_dir)

    for sign in signs:
        # Sanitize folder name
        folder_name = sign.replace(" ", "_").replace("/", "_")
        sign_output_dir = os.path.join(base_output_dir, folder_name)

        # Ensure each sign's folder exists
        if not os.path.exists(sign_output_dir):
            os.makedirs(sign_output_dir)

        # Initialize the Google Image Crawler
        google_crawler = GoogleImageCrawler(storage={'root_dir': sign_output_dir})

        # Start the crawl
        google_crawler.crawl(keyword=sign, max_num=limit, filters=None, file_idx_offset='auto')
        print(f"Downloaded images for {sign} and saved in {sign_output_dir}")

In [ ]:
# Run the download function for the traffic signs
download_traffic_sign_images(traffic_signs)

ERROR:downloader:Response status code 400, file https://media.gettyimages.com/id/976242716/photo/speed-limit-sign-60-km-h.jpg
ERROR:downloader:Response status code 400, file https://media.gettyimages.com/id/1150813637/photo/60-kilometres-per-hour-speed-limit-road-warning-sign.jpg
ERROR:downloader:Response status code 400, file https://media.gettyimages.com/id/465649993/vector/german-road-sign-speed-limit-60-kmh.jpg
ERROR:downloader:Response status code 400, file https://media.gettyimages.com/id/1481245118/photo/road-sign.jpg


Downloaded images for Bangladesh Speed Limit 60 km/h sign on road in Bangladesh and saved in /content/gdrive/MyDrive/VideoToImage_TrafficSign/traffic_signs_images/Bangladesh_Speed_Limit_60_km_h_sign_on_road_in_Bangladesh


ERROR:downloader:Response status code 404, file https://upload.wikimedia.org/wikipedia/commons/thumb/b/bc/Road_sign_in_dhaka_3.jpg
ERROR:downloader:Response status code 403, file https://www.researchgate.net/publication/4376301/figure/fig1/AS:279955654103061@1443758037299/Standard-road-signs-with-Bangla-script-panel.png
ERROR:downloader:Response status code 403, file https://www.researchgate.net/profile/Mahmudul-Hasan-60/publication/224157461/figure/fig1/AS:452634499915781@1484927882722/Some-of-the-Mandatory-traffic-signs-in-Bangladesh_Q320.jpg


Downloaded images for Bangladesh School traffic sign on road in Bangladesh and saved in /content/gdrive/MyDrive/VideoToImage_TrafficSign/traffic_signs_images/Bangladesh_School_traffic_sign_on_road_in_Bangladesh


ERROR:downloader:Response status code 404, file https://upload.wikimedia.org/wikipedia/commons/thumb/b/bc/Road_sign_in_dhaka_3.jpg
ERROR:downloader:Exception caught when downloading file https://w1.png, error: HTTPSConnectionPool(host='w1.png', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x77fcef50a8f0>: Failed to resolve 'w1.png' ([Errno -2] Name or service not known)")), remaining retry times: 2
ERROR:downloader:Exception caught when downloading file https://w1.png, error: HTTPSConnectionPool(host='w1.png', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x77fcef50aa40>: Failed to resolve 'w1.png' ([Errno -2] Name or service not known)")), remaining retry times: 1
ERROR:downloader:Exception caught when downloading file https://w1.png, error: HTTPSConnectionPool(host='w1.png', port=443): Max retries exceeded with url: / (Caused by Name

...

Downloaded images for Bangladesh Road Hump traffic sign on road in Bangladesh and saved in /content/gdrive/MyDrive/VideoToImage_TrafficSign/traffic_signs_images/Bangladesh_Road_Hump_traffic_sign_on_road_in_Bangladesh


ERROR:downloader:Response status code 404, file https://upload.wikimedia.org/wikipedia/commons/thumb/b/bc/Road_sign_in_dhaka_3.jpg


Downloaded images for Bangladesh No Use of Horn traffic sign on road in Bangladesh and saved in /content/gdrive/MyDrive/VideoToImage_TrafficSign/traffic_signs_images/Bangladesh_No_Use_of_Horn_traffic_sign_on_road_in_Bangladesh


ERROR:downloader:Response status code 403, file https://www.researchgate.net/publication/381608451/figure/fig2/AS:11431281254030562@1719027721676/Various-Types-of-Bangladesh-Traffic-Signs.jpg


Downloaded images for Bangladesh Crossroads traffic sign on road in Bangladesh and saved in /content/gdrive/MyDrive/VideoToImage_TrafficSign/traffic_signs_images/Bangladesh_Crossroads_traffic_sign_on_road_in_Bangladesh
